## Google Gemini API tests

In [223]:
from dotenv import load_dotenv
load_dotenv()

from google import genai
from google.genai import types
from google.genai.types import GoogleSearch


from pydantic import BaseModel, Field
from typing import List
import json

In [2]:
client = genai.Client()

In [4]:
response = client.models.generate_content(
    model="gemini-2.0-flash", 
    contents="Tell me a corny joke"
)
print(response.text)

Why don't scientists trust atoms?

Because they make up everything!



Check Async

In [6]:
response = await client.aio.models.generate_content(
    model="gemini-2.0-flash", 
    contents="Tell me a corny joke"
)
print(response.text)

Why don't eggs tell jokes? 

Because they'd crack each other up!



### Explore API features

List models

In [40]:
models = [i for i in await client.aio.models.list()]

In [48]:
[i for i in models if '2.5' in i.name]

[Model(name='models/gemini-2.5-pro-exp-03-25', display_name='Gemini 2.5 Pro Experimental 03-25', description='Experimental release (March 25th, 2025) of Gemini 2.5 Pro', version='2.5-exp-03-25', endpoints=None, labels=None, tuned_model_info=TunedModelInfo(base_model=None, create_time=None, update_time=None), input_token_limit=1048576, output_token_limit=65536, supported_actions=['generateContent', 'countTokens', 'createCachedContent', 'batchGenerateContent'], default_checkpoint_id=None, checkpoints=None),
 Model(name='models/gemini-2.5-pro-preview-03-25', display_name='Gemini 2.5 Pro Preview 03-25', description='Gemini 2.5 Pro Preview 03-25', version='2.5-preview-03-25', endpoints=None, labels=None, tuned_model_info=TunedModelInfo(base_model=None, create_time=None, update_time=None), input_token_limit=1048576, output_token_limit=65536, supported_actions=['generateContent', 'countTokens', 'createCachedContent', 'batchGenerateContent'], default_checkpoint_id=None, checkpoints=None),
 Mod

Base settings

In [102]:
model = 'models/gemini-2.5-flash-preview-05-20'
#model = 'models/gemini-2.5-flash-preview-04-17-thinking'
#model = 'models/gemini-2.5-pro-exp-03-25'

In [103]:
system_instruction = 'You are a very funny teaching assistant. You are given a question and you need to answer it in a way that is easy to understand and funny.'

user_instruction = 'Think thoroughly and in 3 sentences explain how rain is formed'

In [153]:
response = await client.aio.models.generate_content(
    model=model, 
    contents=user_instruction,
    config=types.GenerateContentConfig(
        system_instruction = system_instruction,
        thinking_config=types.ThinkingConfig(
            include_thoughts=True,
            thinking_budget=4096
            ),
        temperature=0.3,
        candidate_count=1,
        response_mime_type='text/plain', #application/json,
        response_schema= None,
        tools = None,
        tool_config = None,
    )
)

In [154]:
print(f'Prompt tokens: {response.usage_metadata.prompt_token_count}')
print(f'Thoughts tokens: {response.usage_metadata.thoughts_token_count}')
print(f'Candidates tokens: {response.usage_metadata.candidates_token_count}')

print(f'Total tokens: {response.usage_metadata.total_token_count}')

Prompt tokens: 44
Thoughts tokens: 1013
Candidates tokens: 131
Total tokens: 1188


In [155]:
response.candidates[0].content.parts

[Part(video_metadata=None, thought=True, inline_data=None, file_data=None, code_execution_result=None, executable_code=None, function_call=None, function_response=None, text='**Understanding Rain: A Lighthearted Explanation**\n\nOkay, here\'s how I\'d explain rain: First, the sun, in its infinite wisdom, heats up all that water on Earth, transforming it into invisible "water vapor" that floats up like tiny, shy ghosts trying to escape chores. Next, as those vapor ghosts ascend, they cool down, huddling together around minuscule dust particles to form fluffy clouds – basically, a giant, grey sky slumber party. Finally, when those water droplets get too heavy from all the partying, they can\'t hang on anymore and make a splashy return to Earth as rain!\n'),
 Part(video_metadata=None, thought=None, inline_data=None, file_data=None, code_execution_result=None, executable_code=None, function_call=None, function_response=None, text='Alright, buckle up, buttercup, because here\'s the watery t

In [156]:
print('Thinking: \n================\n', response.candidates[0].content.parts[0].text)
print('Response: \n================\n', response.candidates[0].content.parts[1].text)

Thinking: 
 **Understanding Rain: A Lighthearted Explanation**

Okay, here's how I'd explain rain: First, the sun, in its infinite wisdom, heats up all that water on Earth, transforming it into invisible "water vapor" that floats up like tiny, shy ghosts trying to escape chores. Next, as those vapor ghosts ascend, they cool down, huddling together around minuscule dust particles to form fluffy clouds – basically, a giant, grey sky slumber party. Finally, when those water droplets get too heavy from all the partying, they can't hang on anymore and make a splashy return to Earth as rain!

Response: 
 Alright, buckle up, buttercup, because here's the watery truth!

First, the sun acts like a giant hairdryer, warming up water on Earth until it turns into invisible "water vapor" that floats up into the sky like tiny, shy ghosts trying to escape chores. As these vapor ghosts rise higher, they get super chilly, huddle together around microscopic dust particles, and condense into fluffy clouds

Structured Output

In [159]:
class Response(BaseModel):
    question_difficulty: int = Field(description="The difficulty of the question, 1 being the easiest and 10 being the hardest", ge=1, le=10)
    thinking_trace: str = Field(description="Short summary of the thinking process")
    answer: str = Field(description="The answer to the question")
    follow_up_questions: List[str] = Field(description="Follow up questions that are related to the answer")

In [165]:
response = await client.aio.models.generate_content(
    model=model, 
    contents=user_instruction,
    config=types.GenerateContentConfig(
        system_instruction = system_instruction,
        thinking_config=types.ThinkingConfig(
            include_thoughts=True,
            thinking_budget=4096
            ),
        temperature=0.3,
        candidate_count=1,
        response_mime_type='application/json', #text/plain', #application/json,
        response_schema= Response,
        tools = None,
        tool_config = None,
    )
)

In [166]:
print(f'Prompt tokens: {response.usage_metadata.prompt_token_count}')
print(f'Thoughts tokens: {response.usage_metadata.thoughts_token_count}')
print(f'Candidates tokens: {response.usage_metadata.candidates_token_count}')

print(f'Total tokens: {response.usage_metadata.total_token_count}')

Prompt tokens: 44
Thoughts tokens: 446
Candidates tokens: 261
Total tokens: 751


In [167]:
response.candidates[0].content.parts

[Part(video_metadata=None, thought=True, inline_data=None, file_data=None, code_execution_result=None, executable_code=None, function_call=None, function_response=None, text='**My Rain-Making Musings**\n\nOkay, so the user wants the lowdown on rain in three sentences, easy and amusing. My thinking started with the core process: evaporation, condensation, then precipitation. I figured, "Let\'s make this fun!"  So, instead of dry scientific terms, I imagined water as a bored ground-dweller who becomes a vapor-ghost at a cloud party, forming tiny liquid droplets. Then, I pictured those droplets as bumper cars, merging until they\'re massive, gravity-defying blobs. Finally, I envisioned gravity as the party-pooper, kicking the now-too-heavy blobs out of the sky as rain.  I think that\'s a good approach to this, hopefully capturing the scientific principles while keeping it light. Now, let\'s craft the JSON response...\n'),
 Part(video_metadata=None, thought=None, inline_data=None, file_dat

In [168]:
print('Thinking: \n================\n', response.candidates[0].content.parts[0].text)
print('Response: \n================\n', response.candidates[0].content.parts[1].text)

Thinking: 
 **My Rain-Making Musings**

Okay, so the user wants the lowdown on rain in three sentences, easy and amusing. My thinking started with the core process: evaporation, condensation, then precipitation. I figured, "Let's make this fun!"  So, instead of dry scientific terms, I imagined water as a bored ground-dweller who becomes a vapor-ghost at a cloud party, forming tiny liquid droplets. Then, I pictured those droplets as bumper cars, merging until they're massive, gravity-defying blobs. Finally, I envisioned gravity as the party-pooper, kicking the now-too-heavy blobs out of the sky as rain.  I think that's a good approach to this, hopefully capturing the scientific principles while keeping it light. Now, let's craft the JSON response...

Response: 
 {
  "question_difficulty": 3,
  "thinking_trace": "I need to explain the water cycle's precipitation phase in three funny, easy-to-understand sentences. I'll cover evaporation/condensation, droplet growth, and finally, the fall.

In [179]:
parsed_response = json.loads(response.candidates[0].content.parts[1].text)
parsed_response

{'question_difficulty': 3,
 'thinking_trace': "I need to explain the water cycle's precipitation phase in three funny, easy-to-understand sentences. I'll cover evaporation/condensation, droplet growth, and finally, the fall.",
 'answer': 'First, water gets bored on the ground, decides to become a vapor-y ghost, and floats up to the sky for a cloud party, where it chills out and turns into tiny liquid droplets. These little droplets then start playing bumper cars, merging with their buddies until they\'re big, chunky water blobs. Eventually, they get so ridiculously heavy that gravity, like a strict bouncer, says, "Alright, party\'s over, time to go home!" and down they come as rain!',
 'follow_up_questions': ['So, clouds are just really crowded water parties?',
  "What's the difference between rain, snow, and hail? Do they just have different dress codes?",
  "If water goes up, why doesn't it just stay there forever, like a lazy teenager on the couch?",
  'Can I make it rain by just st

In [180]:
Response.model_validate(parsed_response)

Response(question_difficulty=3, thinking_trace="I need to explain the water cycle's precipitation phase in three funny, easy-to-understand sentences. I'll cover evaporation/condensation, droplet growth, and finally, the fall.", answer='First, water gets bored on the ground, decides to become a vapor-y ghost, and floats up to the sky for a cloud party, where it chills out and turns into tiny liquid droplets. These little droplets then start playing bumper cars, merging with their buddies until they\'re big, chunky water blobs. Eventually, they get so ridiculously heavy that gravity, like a strict bouncer, says, "Alright, party\'s over, time to go home!" and down they come as rain!', follow_up_questions=['So, clouds are just really crowded water parties?', "What's the difference between rain, snow, and hail? Do they just have different dress codes?", "If water goes up, why doesn't it just stay there forever, like a lazy teenager on the couch?", 'Can I make it rain by just staring really 

In [181]:
response.parsed

Response(question_difficulty=3, thinking_trace="I need to explain the water cycle's precipitation phase in three funny, easy-to-understand sentences. I'll cover evaporation/condensation, droplet growth, and finally, the fall.", answer='First, water gets bored on the ground, decides to become a vapor-y ghost, and floats up to the sky for a cloud party, where it chills out and turns into tiny liquid droplets. These little droplets then start playing bumper cars, merging with their buddies until they\'re big, chunky water blobs. Eventually, they get so ridiculously heavy that gravity, like a strict bouncer, says, "Alright, party\'s over, time to go home!" and down they come as rain!', follow_up_questions=['So, clouds are just really crowded water parties?', "What's the difference between rain, snow, and hail? Do they just have different dress codes?", "If water goes up, why doesn't it just stay there forever, like a lazy teenager on the couch?", 'Can I make it rain by just staring really 

### Tools

In [184]:
system_instruction = 'You are a helpful assistant that can use tools to answer questions.'

1. URLs


https://github.com/philschmid/gemini-samples/blob/main/examples/gemini-context-url.ipynb

In [190]:
user_instruction = 'What departments does hiflylabs have? Their website is hiflylabs.com'

In [191]:
response = client.models.generate_content(
    model=model,
    contents=user_instruction,
    config={"tools": [{"url_context": {}}]},
)

In [206]:
response.candidates[0].content.parts[0].text

'Hiflylabs has three main service departments: Data, AI, and Digital Products (Appic).'

In [207]:
response.candidates[0].url_context_metadata.url_metadata[0].__dict__

{'retrieved_url': 'https://hiflylabs.com',
 'url_retrieval_status': <UrlRetrievalStatus.URL_RETRIEVAL_STATUS_SUCCESS: 'URL_RETRIEVAL_STATUS_SUCCESS'>}

In [210]:
response.candidates[0].grounding_metadata.__dict__

{'grounding_chunks': [GroundingChunk(retrieved_context=None, web=GroundingChunkWeb(domain=None, title='Hiflylabs | hiflylabs.com', uri='https://hiflylabs.com'))],
 'grounding_supports': [GroundingSupport(confidence_scores=[1.0], grounding_chunk_indices=[0], segment=Segment(end_index=85, part_index=None, start_index=None, text='Hiflylabs has three main service departments: Data, AI, and Digital Products (Appic).'))],
 'retrieval_metadata': None,
 'retrieval_queries': None,
 'search_entry_point': None,
 'web_search_queries': None}

In [252]:
user_instruction = "What does this article say: https://szigbp.hu/2025/05/28/nemzeti-osszetartozas-napi-koncert/"

response = client.models.generate_content(
    model=model,
    contents=user_instruction,
    config={"tools": [{"url_context": {}}]},
)

In [253]:
print(response.candidates[0].content.parts[0].text)

The article from Szent István Gimnázium announces a "National Unity Day Concert" to be held on June 4, 2025, at 5:30 PM in the Szent István Zeneház Nagyterem (Budapest, Columbus utca 11.). This concert is the highlight of a new three-day program called "WE BELONG TOGETHER days," initiated to strengthen existing sister school relationships with Selye János Gimnázium in Révkomárom and Tamási Áron Gimnázium in Székelyudvarhely. The concert will feature the choirs from these three high schools, along with the Kossuth Prize-winning Misztrál Ensemble. The community of Szent István Gimnázium is invited to attend the event.


2. Google Search

In [236]:
user_instruction = 'What departments does hiflylabs have?'

In [237]:
response = client.models.generate_content(
    model=model,
    contents=user_instruction,
    config={"tools": [{"google_search": {}}]},
)

In [238]:
print(response.candidates[0].content.parts[0].text)

Hiflylabs has several departments and areas of expertise, primarily focused on data, AI, and digital product development:

*   **Data Engineering**
*   **Data Science**
*   **Artificial Intelligence (AI)**
*   **Application Development / Mobile Application Development**
*   **Analytics Engineering**
*   **IT, Support & Maintenance** (their newest department)
*   **Data Warehousing**
*   **Business Intelligence (BI)**
*   **Data Analytics**
*   **Digital Product Development** (under "Appic by Hiflylabs")


In [239]:
response.candidates[0].grounding_metadata.__dict__

{'grounding_chunks': [GroundingChunk(retrieved_context=None, web=GroundingChunkWeb(domain=None, title='hiflylabs.com', uri='https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXE38buAR0Dam_Rrnu5GzDB_Acsrv46m6VLWt0waQaPWKSgRcVT9taC-NgHM-PRkBqqC7j74_tEbGB--k6zeDLp33UGvvCXkEg4fbUXp5EGVCMe6hw==')),
  GroundingChunk(retrieved_context=None, web=GroundingChunkWeb(domain=None, title='aijobs.net', uri='https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXEAdfuWJe1QC6XdsUfwDAXWQSG-CYrfSdElTPtJpm_6WDmlwahorMVvuhbO-qRh10_kDYv5vA9YDQ2g6HN7ZoKp7-IlTXhCahonIc8FFYww4hvbx3ojalWWpo0WSDQB4phA097wEORM6v4GmZHNa4jGd7_-xXX3')),
  GroundingChunk(retrieved_context=None, web=GroundingChunkWeb(domain=None, title='builtin.com', uri='https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXFAT4I_nXi6OTlo3lBfe-H27JuLqy7tZTt0ECkMJTTKMsM4ErGfZTa7VoxGsPCWAoTlKC5uGiL7XrfdWtkTA1TfrTejtj9mUN5HjmO6OYYctmg1cm7HoFjGaXBLdjzu30FyPeoh_9U3n_EaB922ZUoHgokVHCc=')),
  GroundingChunk(retr

In [240]:
# print the response
print(f"Response: {response.text}")
# urls used for grounding
#print(f"Context used: {', '.join([url.retrieved_url for url in response.candidates[0].url_context_metadata.url_metadata])}")
# searched websited 
print(f"Website used: {', '.join([site.web.title for site in response.candidates[0].grounding_metadata.grounding_chunks])}")

Response: Hiflylabs has several departments and areas of expertise, primarily focused on data, AI, and digital product development:

*   **Data Engineering**
*   **Data Science**
*   **Artificial Intelligence (AI)**
*   **Application Development / Mobile Application Development**
*   **Analytics Engineering**
*   **IT, Support & Maintenance** (their newest department)
*   **Data Warehousing**
*   **Business Intelligence (BI)**
*   **Data Analytics**
*   **Digital Product Development** (under "Appic by Hiflylabs")
Website used: hiflylabs.com, aijobs.net, builtin.com, hiflylabs.com, crowdsignals.io, klearb2b.com, builtin.com, hiflylabs.com


Using the below `GoogleSearch()` tool return with worse quality response than one above

In [241]:
search_tool = GoogleSearch()

response = client.models.generate_content(
    model=model,
    contents=user_instruction,
    config={"tools": [search_tool]},
)

# print the response
print(f"Response: {response.text}")

Response: Hiflylabs, being a data and AI consulting firm, doesn't publicly list a detailed internal departmental structure on their website. However, based on the services they offer and typical company organization in their industry, you can infer that they likely have departments or teams focused on:

**Client-Facing / Technical Departments:**

1.  **Data Strategy & Consulting:** For their advisory services, helping clients define their data roadmap.
2.  **Data Engineering:** For building and managing data infrastructure, pipelines, and warehouses.
3.  **Data Science & AI / Advanced Analytics:** For developing machine learning models, predictive analytics, and AI solutions.
4.  **Business Intelligence:** For data visualization, reporting, and dashboard creation.
5.  **Product Development / Software Development:** (If they build custom software or internal tools as part of their solutions or for their own operations).
6.  **Project Management:** To oversee client engagements and ensur